Installing dependencies

In [39]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

skipping


Trying out TradingView API

In [40]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [52]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import requests
from binance.client import Client

In [50]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']
ta_api = os.environ['taapi_key']


# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
# info = client.get_symbol_info('ETHUSDT')
# print(info)

In [43]:
def get_data_frame(symbol):
    starttime = '3 days ago UTC'
    interval = '15m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

,date,open,high,low,close
0,1667049300000,1622.19000000,3287.09000000,1608.69000000,1617.85000000
1,1667050200000,1617.53000000,1619.83000000,1609.25000000,1611.20000000
2,1667051100000,1610.97000000,1625.13000000,1545.00000000,1624.62000000
3,1667052000000,1624.43000000,1640.49000000,1621.03000000,1632.92000000
4,1667052900000,1632.84000000,1643.98000000,1627.28000000,1629.46000000
...,...,...,...,...,...
283,1667304000000,1590.29000000,1599.25000000,1585.79000000,1590.88000000
284,1667304900000,1591.00000000,1595.26000000,1590.12000000,1593.85000000
285,1667305800000,1593.92000000,2500.00000000,1592.26000000,1593.88000000
286,1667306700000,1593.89000000,1598.32000000,1593.53000000,1597.29000000


In [44]:
def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

,date,open,high,low,close,macd,signal,200_ema,Trigger
0,2022-10-29 13:15:00,1622.19,3287.09,1608.69,1617.85,0.000000,0.000000,1617.850000,0
1,2022-10-29 13:30:00,1617.53,1619.83,1609.25,1611.20,-0.530484,-0.106097,1617.783831,0
2,2022-10-29 13:45:00,1610.97,1625.13,1545.00,1624.62,0.130481,-0.058781,1617.851852,1
3,2022-10-29 14:00:00,1624.43,1640.49,1621.03,1632.92,1.308952,0.214765,1618.001784,1
4,2022-10-29 14:15:00,1632.84,1643.98,1627.28,1629.46,1.941328,0.560078,1618.115796,1
...,...,...,...,...,...,...,...,...,...
283,2022-11-01 12:00:00,1590.29,1599.25,1585.79,1590.88,21.505690,0.764518,1568.974424,1
284,2022-11-01 12:15:00,1591.00,1595.26,1590.12,1593.85,23.513707,5.314356,1569.221942,1
285,2022-11-01 12:30:00,1593.92,2500.00,1592.26,1593.88,24.821371,9.215759,1569.467296,1
286,2022-11-01 12:45:00,1593.89,1598.32,1593.53,1597.29,25.835053,12.539618,1569.744139,1


# BACKTESTING

Backtesting completed on 6 month historical data on 15m time frame

In [45]:
def get_six_month_data_frame(symbol):
    starttime = '6 months ago UTC'
    interval = '15m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
    return df

six_month_eth_df_raw = get_six_month_data_frame('ETHUSDT')

In [46]:
six_month_eth_df_raw

,date,open,high,low,close
0,1664969400000,1334.55000000,1334.55000000,1334.20000000,1334.20000000
1,1664970300000,1334.00000000,1334.00000000,1331.21000000,1331.21000000
2,1664971200000,1334.19000000,4277.48000000,1069.37000000,4277.48000000
3,1664972100000,4277.48000000,4277.48000000,4277.48000000,4277.48000000
4,1664973000000,1331.64000000,4277.48000000,1328.61000000,1335.00000000
...,...,...,...,...,...
2594,1667304000000,1590.29000000,1599.25000000,1585.79000000,1590.88000000
2595,1667304900000,1591.00000000,1595.26000000,1590.12000000,1593.85000000
2596,1667305800000,1593.92000000,2500.00000000,1592.26000000,1593.88000000
2597,1667306700000,1593.89000000,1598.32000000,1593.53000000,1597.29000000


In [47]:
def test_macd_logic(symbol):

    symbol_df = get_six_month_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['close'] > symbol_df['200_ema']) & (symbol_df['macd'] < 0) & (symbol_df['signal'] > symbol_df['macd'].shift())), 1, 0)

    symbol_df = symbol_df.iloc[200:]

    triggered_df = symbol_df.loc[symbol_df['Trigger'] == 1]

    return symbol_df

test_df = test_macd_logic('ETHUSDT')

In [49]:
in_trade = False

win = 0
loss = 0

for index, row in test_df.iterrows():
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True
            
    
    else:

        
    # print(row['open'], row['close'], index)

date       2022-10-07 13:30:00
open                   1331.17
high                   1336.45
low                    1290.53
close                  1331.56
macd                 -5.763268
signal               -2.963003
200_ema            1361.157678
Trigger                      0
Name: 200, dtype: object
date       2022-10-07 13:45:00
open                   1331.52
high                   1333.42
low                    1329.59
close                  1332.83
macd                  -6.09141
signal               -3.588684
200_ema            1360.875811
Trigger                      0
Name: 201, dtype: object
date       2022-10-07 14:00:00
open                   1332.17
high                   1338.63
low                    1331.11
close                  1337.53
macd                 -5.904154
signal               -4.051778
200_ema            1360.643514
Trigger                      0
Name: 202, dtype: object
date       2022-10-07 14:15:00
open                   1337.52
high                    13

In [54]:
indicator = "atr"
endpoint = f"https://api.taapi.io/{indicator}"
parameters = {
    'secret': ta_api,
    'exchange': 'binance',
    'symbol': 'ETH/USDT',
    'interval': '15m'
    } 
response = requests.get(url = endpoint, params = parameters)
result = response.json() 
print(result)

{'value': 8.230954805762398}
